# Nano-GPT producing Shakespeare text

In [ ]:
# Import all the necessary libraries
import time
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 64                       # number of independent sequences in text
block_size = 256                      # maximum context length of prediction
max_iters = 3000                      # number of iteration
eval_interval = 500                   # interval for evaluation
learning_rate = 3e-4                  # learning rate for optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200                      # number of iteration for evaluation
n_embd = 384                          # number of embedding dimensions --> for one word
n_head = 6                            # number of heads in one attention block --> small parallel layers in one block
n_layer = 6                           # number of different self-attention blocks all together
dropout = 0.2                         # dropout probability

In [ ]:
torch.manual_seed(1337)   # random number seed


In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)  # size of the vocabulary

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]           # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]  # 90% for training
val_data = data[n:]    # 10% for validation

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
# gives a better estimate of the loss function by nullifying the noise
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()                         # sets model to evaluation phase
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()                       # sets model to training phase
    return out

In [ ]:
# one Head of each Self-Attention block
class Head(nn.Module):
    def __init__(self, head_size):                              # head_size = n_embed // n_head
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)     # n_embed --> head_size
        self.query = nn.Linear(n_embd, head_size, bias=False)   # n_embed --> head_size
        self.value = nn.Linear(n_embd, head_size, bias=False)   # n_embed --> head_size
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch_size, block_size(T), channels(vocab_size=65))
        # output of size (batch_size, block_size, head_size)
        B,T,C = x.shape
        k = self.key(x)                                         # (batch_size, block_size, head_size)
        q = self.query(x)                                       # (batch_size, block_size, head_size)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5        # (batch_size, block_size, head_size) @ (batch_size, head_size, block_size) --> (batch_size, block_size, block_size)
        # so here for a specific batch, we multiply (block_size, head_size) @ (head_size, block_size) --> (block_size, block_size)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (batch_size, block_size, block_size)
        wei = F.softmax(wei, dim=-1)                            # (batch_size, block_size, block_size)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)                                       # (batch_size, block_size, head_size)
        out = wei @ v                                           # (batch_size, block_size, block_size) @ (batch_size, block_size, head_size) --> (batch_size, block_size, head_size)
        return out

In [ ]:
#  multiple self-attention heads in parallel
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # different heads
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # concatenate h for all heads vertically
        # --> dimension = head_size*num_heads (because of each head * because of many heads)
        out = self.dropout(self.proj(out))                   # (batch_size, block_size, n_embd)  --> because of projection
        return out

In [ ]:
# a simple feedfprward layer
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),                   # n_embed --> 4*n_embed  (according to transformers paper)
            nn.ReLU(),                                       # ReLU activation function for non-linearity
            nn.Linear(4 * n_embd, n_embd),                   # 4*n_embed --> n_embed  (according to transformers paper)
            nn.Dropout(dropout),                             # dropout some connections with prob = dropout
        )

    def forward(self, x):
        return self.net(x)                                   # create the neural net

In [ ]:
# one single block containing self-attention, feed forward, layer-norm etc etc
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head                         # n_embd = head_size*n_head
        self.sa = MultiHeadAttention(n_head, head_size)      # self-attention
        self.ffwd = FeedFoward(n_embd)                       # feed-forward network
        self.ln1 = nn.LayerNorm(n_embd)                      # layer-norm
        self.ln2 = nn.LayerNorm(n_embd)                      # layer-norm

    def forward(self, x):
        x = x + self.sa(self.ln1(x))                         # residual connection
        x = x + self.ffwd(self.ln2(x))                       # residual connection
        return x

In [ ]:
# the main language model class having numto
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)           # vocab_size --> n_embed
        self.position_embedding_table = nn.Embedding(block_size, n_embd)        # block_size --> n_embed (because positional embeddings)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # multiple blocks in sequence with each block having self-attention, feed-forwaed, layer-norm etc
        self.ln_f = nn.LayerNorm(n_embd)                                        # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)                            # final linear layer
        self.apply(self._init_weights)                                          # initialize the weights

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape                                                        # batch_size, block_size
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)                               # (batch_size, block_size, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (block_size, n_embed)
        x = tok_emb + pos_emb                                                   # (batch_size, block_size, n_embed) via brodcasting
        x = self.blocks(x)                                                      # (batch_size, block_size, n_embed)
        x = self.ln_f(x)                                                        # (batch_size, block_size, n_embed)
        logits = self.lm_head(x)                                                # (batch_size, block_size, channels (vocab_size=65))

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]                                     # crop idx to the last block_size tokens
            logits, loss = self(idx_cond)                                       # get the predictions
            logits = logits[:, -1, :]                                           # focus only on the last time step --> becomes (B, C)
            probs = F.softmax(logits, dim=-1)                                   # apply softmax to get probabilities --> still (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)                  # sample from the distribution --> becomes (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)                             # append sampled index to the running sequence --> (B, T+1)
        return idx

In [ ]:
# call the model
model = GPTLanguageModel()
m = model.to(device)

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [ ]:
# start the timer
training_start_time = time.time()

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2221, val loss 4.2306
step 500: train loss 1.7600, val loss 1.9146
step 1000: train loss 1.3903, val loss 1.5987
step 1500: train loss 1.2644, val loss 1.5271
step 2000: train loss 1.1835, val loss 1.4978
step 2500: train loss 1.1233, val loss 1.4910
step 2999: train loss 1.0733, val loss 1.4809


In [ ]:
# stop the timer
training_stop_time = time.time()

In [ ]:
print(f'Time taken for training: {training_stop_time - training_start_time}')

Time taken for training: 1955.1144347190857


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)        # kickstart the generation by choosing tensor([[0]])

#### Change the `max_new_tokens` to generate larger or smaller paragraphs

In [ ]:
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))    # generate the text using context


Himself, being drument, or is nother, and then.

KING RICHARD II:
Go, let us touch of Lycusgh, busin English doth:
Then, faee sinciate, enough they with so:
These fasts that take down the little love.

QUEEN ELIZABETH:
The trumph since, fair suns; and hasted, way, burn her!

KING RICHARD III:
In sure, I have side. Now, granted musicians!
Come, good cousin, Claudio.

DUKE OF YORK:
Cry, madam: I'll marry hand my cousin.

DUKE OF YORK:

KING RICHARD III:

CATESBY:
My queen Clarence is; and I muster shall advise you
To you love my body's highnesst.

EDWARD:
His paint I swoon? and their marks now, since--

KING EDNARY ?
RICHARDARD:
How will they stare go to have witne here
Six you, look free now it.

ISABELLA:
See, I would for this the house you might of these
Of what of her marriage:
O you talk now, sir, and underately
Your lord I did put here I left in ironate-to-like
A stay attance in perck doul! 't be no guire.

ANTIJ IITA:
Go-more day, sir; he will make a chidren work:
I have told him